<a href="https://colab.research.google.com/github/Kishore8949/LLMsApplications/blob/main/Finetune_falcon_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tune Falcon-7b-instruct-sharded model on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face. Links to both the model and dataset are in the notebook.

# Install and Imports

In [5]:
#install dependencies
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
notebook_login()

# Loading data from hugging face

In [4]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
data = load_dataset(dataset_name)
data

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

In [5]:
from datasets import DatasetDict

In [6]:
print(data.keys())

dict_keys(['train'])


In [7]:
features = data['train']

In [8]:
print(features['text'][0])

<HUMAN>: What is a panic attack?
<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.


# Loading the model and Setting up bitsandbytes config

In [6]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Setting up the LoRA config

In [8]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


# Load the trainer

In [12]:
output_dir = "falcon7binstruct_mentalhealthmodel"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #180 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

# Passing arguments to the SFTT trainer

In [13]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [14]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

# Train the model

In [15]:
%%time
peft_model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.706100
20,1.343200
30,0.962300
40,0.469000
50,0.153400
60,0.057800
70,0.043200
80,0.038600
90,0.036500
100,0.035200


CPU times: user 2h 12min 40s, sys: 42 s, total: 2h 13min 22s
Wall time: 2h 18min 43s


TrainOutput(global_step=180, training_loss=0.28414463814761903, metrics={'train_runtime': 8309.8863, 'train_samples_per_second': 1.386, 'train_steps_per_second': 0.022, 'total_flos': 9.545509310439629e+16, 'train_loss': 0.28414463814761903, 'epoch': 65.45})

# Save the model

In [22]:
# trainer.save() #if you want to save your model locally

In [17]:
trainer.push_to_hub() #push to hub

CommitInfo(commit_url='https://huggingface.co/Kishore098/falcon7binstruct_mentalhealthmodel/commit/d14e0773759a7f6b1da5964c424af589f3e563f2', commit_message='End of training', commit_description='', oid='d14e0773759a7f6b1da5964c424af589f3e563f2', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [9]:
# Loading PEFT model
PEFT_MODEL = "Srishy/falcon7binstruct_mentalhealthmodel_oct23"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [11]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [12]:
get_response("Are there cures for mental health problems?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  ###Option 1: Yes, there are cures for mental health problems.

  ###Option 2: No, there are no cures for mental health problems.

  ###Option 3: It depends on the individual case.

  ###Option 4: It depends on the individual case.

  ###Option 5: It depends on the individual case.

  ###Option 6: It depends on the individual case.

  ###Option 7: It depends on the individual case.

  ###Option 8: It depends on the individual case.

  ###Option 9: It depends on the individual case.

  ###Option 10: It depends on the individual case.

  ###Option 11: It depends on the individual case.

  ###Option 12: It depends on the individual case.

  ###Option 13: It depends on the indi

In [13]:
get_response("What’s the difference between psychotherapy and counselling?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What’s the difference between psychotherapy and counselling?

  ###Response:

  ###Therapy is a broad term that can refer to a range of different approaches to mental health treatment. It can include a variety of different techniques, such as cognitive behavioral therapy, psychodynamic therapy, and mindfulness-based therapies. Counseling, on the other hand, is a more specific term that generally refers to a shorter-term, goal-oriented approach that focuses on helping individuals achieve specific goals or improve their relationships. It may involve techniques from both therapy and counseling, depending on the individual's needs.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

# Zero shot prompting

In [21]:
# Generate sentiment analysis from both orignal model and fine-tuned model
def get_sentiment_zero_shot(sentence):
  prompt = f"""

  Instruction: Classify the text into positive, neutral or negative:

  Text: {sentence}

  Classification:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  sentiment = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{sentiment}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_sentiment = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_sentiment}')


In [22]:
get_sentiment_zero_shot("Mental Health is a disease")

Response from original falcon_7b_instruct_sharded:


  Instruction: Classify the text into positive, neutral or negative:

  Text: Mental Health is a disease

  Classification:

  - Positive
  - Neutral
  - Negative
User 
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:


  Instruction: Classify the text into positive, neutral or negative:

  Text: Mental Health is a disease

  Classification:

   Positive: The text talks about how diseases of the brain can be treated with medication.
   Neutral: The text discusses a common medical condition.
   Negative: The text portrays mental health as a disease which many people are of the opinion that it is a false diagnosis made up by doctors to sell more drugs.

It's essential to remember that this classification might need to be revised over time as my AI language model isn't perfect yet. If you have any feedback or suggestions to improve it, please don't hesitate to let me know. 

Ta

# Few shot prompting

In [19]:
# Generate sentiment analysis from both orignal model and fine-tuned model
def get_sentiment_few_shot(sentence):
  prompt = f"""
  Instruction: Classify the input text into positive, neutral or negative:

  Labelled Examples:
  Text: Today the weather is fantastic
  Classification: Pos
  Text: The furniture is small.
  Classification: Neu
  Text: I don't like your attitude
  Classification: Neg

  Input Text: {sentence}

  Classification:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  sentiment = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{sentiment}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_sentiment = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_sentiment}')


In [20]:
get_sentiment_few_shot("Mental Health is a disease")

Response from original falcon_7b_instruct_sharded:

  Instruction: Classify the input text into positive, neutral or negative:

  Labelled Examples:
  Text: Today the weather is fantastic
  Classification: Pos
  Text: The furniture is small.
  Classification: Neu
  Text: I don't like your attitude
  Classification: Neg

  Input Text: Mental Health is a disease

  Classification:

  - Positive: Mental Health is a disease
  - Neutral: Mental Health is a disease
  - Negative: Mental Health is a disease

The output of the model is a set of labels that are used to classify the input text. In this case, the labels are "Positive", "Neutral", and "Negative". The model is trained on a dataset of labelled examples, which allows it to learn the patterns in the text that correspond to each label.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  Instruction: Classify the input text into positive, neutral or negative:

  Labelled Example